In [170]:
import pandas as pd
import numpy as np

import spacy
from spacy.lang.id import Indonesian

import nltk
from nltk.corpus import wordnet as wn

nltk.download('wordnet')
nlp = Indonesian()  # use directly
stopwords = spacy.lang.id.stop_words.STOP_WORDS

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [171]:
data = pd.read_csv("dataset.csv")
label_drop = ["web-scraper-order","web-scraper-start-url","pengadu"]
data = data.drop(columns=label_drop)
data.tail()

,diposisi,judul,keluhan,tipe
12856,Kementerian Sosial,Orang-Orang Mampu Mendapatkan BLSM,KPS di daerah desa sambong belum ada BLSM..dan...,Bantuan Langsung Sementara Masyarakat (BLSM)
12857,Kantor Walikota Jakarta Timur,Pungli di Kecamatan Makasar Dalam Pengurusan IMB,Hal melaporkan Staf Perizinan Bangunan Kec. Ma...,Reformasi Birokrasi dan Tata Kelola
12858,Kantor Walikota Jakarta Utara,Kader-kader Kelurahan Cilincing Memanfaatkan P...,Yth. Pemerintah Provinsi DKI Jakarta. Saya ing...,Infrastruktur
12859,Kementerian Sosial,BLSM Di Bondowoso,KPS Di desa kemirian kec. tamanan kab.bondowo...,Bantuan Langsung Sementara Masyarakat (BLSM)
12860,Kementerian Sosial,BLSM Tidak Tepat Sasaran di Mataram,KPS BLSM tdk tepat sasaran mRNA ada yg seharus...,Kepesertaan Non-KPS


In [172]:
# print('\nNumber of missing values:')
# for col in data.columns:
#     print('\t%s: %d' % (col, data[col].isnull().sum()))
    
# # cek jumlah perkelas
# print('\nNumber of class values:')
# print(data.iloc[:,-1].value_counts(dropna=False))
tipe = data["tipe"].to_list()
data["tipe"].value_counts(dropna=False) #155 kelas diposisi, 37 kelas tipe

Bantuan Langsung Sementara Masyarakat (BLSM)                                3592
Kepesertaan Non-KPS                                                         2526
Implementasi Penyesuaian Harga BBM                                           963
Topik Lainnya                                                                806
Beras Miskin (Raskin)                                                        694
Infrastruktur                                                                693
Reformasi Birokrasi dan Tata Kelola                                          643
Kelangkaan BBM                                                               516
Pendidikan                                                                   487
Lingkungan Hidup dan Penanggulangan Bencana                                  413
Kesehatan                                                                    401
Kepesertaan KPS                                                              353
Banjir Jakarta 2013         

In [180]:
# Topic Modelling

def tokenize(teks):
    sentence = nlp(teks)
    token_kata = [token.text for token in sentence]
    
    return token_kata

def lemma(teks):
    for v in teks:
        lemma = [token.lemma_ for token in nlp(v)]
    
    return lemma

def stopword(teks):
    clean = []
    for i in teks:
        if i not in stopwords:
            clean.append(i)
#             return i
    return clean

In [182]:
def preprocessing(text):
    clean = text.lower()
    clean = tokenize(text)
    
    return clean

In [183]:
texts = data["keluhan"].to_list() 
# # lemmas = lemma(token)
# # text = stopword(lemmas)
# # text
# for line in texts:
clean = data["keluhan"].apply(preprocessing)
    
clean

0        [Lampu, Lalu, Lintas, di, Jalan, Juanda, dan, ...
1        [Yth, ., Pemprov, DKI, Jakarta, ., Saya, guru,...
2        [KPS, kecamatan, :, patumbak,,Kab, ., Deli, Se...
3        [Pak, ., Tolong, dong, ,, angkutan, umum, mikr...
4                        [BBM, setuju, naik, demi, rakyat]
5        [KPS, di, daerah, desa, sambong, belum, ada, B...
6        [Hal, melaporkan, Staf, Perizinan, Bangunan, K...
7        [Yth, ., Pemerintah, Provinsi, DKI, Jakarta, ....
8        [KPS,  , Di, desa, kemirian, kec, ., tamanan, ...
9        [KPS, BLSM, tdk, tepat, sasaran, mRNA, ada, yg...
10       [KPS, 888bdt, 56789009, di, desa, /, kampung, ...
11       [BBM, gmn, laporan, pengaduan, saya, mengenai,...
12       [KPS, Rumah, tangga, saya, atas, nama, H.ABDUL...
13       [KPS, DIWILAYAH, RW, 06, KALIDERES, BANYAK, YG...
14       [KPS,  , Tolong, di, tindaklanjuti, di, desa, ...
15       [Yth, ., Pemprov, DKI, Jakarta, ., \n\n\n\n, S...
16       [Tolong, ditinjau, depan, pasar, tanah, abang,.

In [184]:
clean.head()

0    [Lampu, Lalu, Lintas, di, Jalan, Juanda, dan, ...
1    [Yth, ., Pemprov, DKI, Jakarta, ., Saya, guru,...
2    [KPS, kecamatan, :, patumbak,,Kab, ., Deli, Se...
3    [Pak, ., Tolong, dong, ,, angkutan, umum, mikr...
4                    [BBM, setuju, naik, demi, rakyat]
Name: keluhan, dtype: object